In [1]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix


rand_seed = 12


# Preprocessing

In [2]:
# Load the audio file
audio_path = 'data_audio/300_AUDIO.wav'
y, sr = librosa.load(audio_path)

# Extract audio features
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
chroma = librosa.feature.chroma_stft(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)

# Aggregate the features (mean, std, etc.)
mfccs_mean = np.mean(mfccs, axis=1)
chroma_mean = np.mean(chroma, axis=1)
zcr_mean = np.mean(zcr, axis=1)

audio_features = np.concatenate((mfccs_mean, chroma_mean, zcr_mean), axis=0)


In [14]:
# Load COVAREP features
covarep_path = 'data_audio/300_COVAREP.csv'
covarep_features = pd.read_csv(covarep_path)

# Load FORMANT features
formant_path = 'data_audio/300_FORMANT.csv'
formant_features = pd.read_csv(formant_path)

# Combine features
all_features = np.concatenate((audio_features, covarep_features.values.flatten(), formant_features.values.flatten()), axis=0)


[-440.24969482   86.91956329    6.044837   ... 2410.         3773.5
 4734.5       ]


# Exploratory data analysis

In [4]:
# Print the shape of the features
print(f'Audio features shape: {audio_features.shape}')
print(f'COVAREP features shape: {covarep_features.shape}')
print(f'FORMANT features shape: {formant_features.shape}')
print(f'Combined features shape: {all_features.shape}')

Audio features shape: (26,)
COVAREP features shape: (64850, 74)
FORMANT features shape: (64849, 5)
Combined features shape: (5123171,)


# Train test split

In [5]:
# Assume we have a dataset with more samples
# Example: features = np.array([all_features_1, all_features_2, ...])
# Example: labels = np.array([label_1, label_2, ...])

features = np.array([all_features, all_features, all_features])  # Replace with actual feature array
labels = np.array([0, 1, 0])  # Replace with actual labels


# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=rand_seed)


# Creating the pipeline & hyperparameter tuning

In [19]:
from sklearn.model_selection import StratifiedKFold

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier())
])

# Define the hyperparameters
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10]
}

# Use StratifiedKFold for cross-validation
stratified_kf = StratifiedKFold(n_splits=2)

# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=stratified_kf, scoring='accuracy')
grid_search.fit(X_train, y_train)


# Best parameters
best_params = grid_search.best_params_
print(f'Best parameters: {best_params}')


ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

In [ ]:
# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier())
])

# Define the hyperparameters
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10]
}

# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters
best_params = grid_search.best_params_
print(f'Best parameters: {best_params}')


# Evaluation

In [ ]:
# Make predictions
y_pred = grid_search.predict(X_test)

# Evaluation
accuracy = grid_search.score(X_test, y_test)
print(f'Model Accuracy: {accuracy * 100:.2f}%')

# Classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)
